Datos se colocan en Data/modelnet40_ply_hdf5_2048: https://huggingface.co/datasets/Msun/modelnet40/resolve/d5dc795541800feeb7a4b3bd3142729a0d2adf7a/modelnet40_ply_hdf5_2048.zip

In [1]:
import torch
import learning3d
import open3d as o3d
import numpy as np
from tqdm import tqdm

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


Agnostig device

In [3]:
if not torch.cuda.is_available():
    device = 'cpu'
print(device)
device = torch.device(device)

cpu


In [4]:
import os
root_dir = "./"
DATA_DIR = os.path.normpath(os.path.join(root_dir, 'data'))
if not os.path.exists(DATA_DIR):
    os.mkdir(DATA_DIR)
if not os.path.exists(os.path.join(DATA_DIR, 'modelnet40_ply_hdf5_2048')):
    www = 'https://huggingface.co/datasets/Msun/modelnet40/resolve/d5dc795541800feeb7a4b3bd3142729a0d2adf7a/modelnet40_ply_hdf5_2048.zip'
    zipfile = os.path.basename(www)
    os.system('wget --no-check-certificate %s; unzip %s' % (www, zipfile))
    os.system('mv %s %s' % (zipfile[:-4], DATA_DIR))
    os.system('rm %s' % (zipfile))

In [5]:
torch.backends.cudnn.deterministic = True
from torch.utils.data import DataLoader
from learning3d.models import DGCNN, DCP
from learning3d.data_utils import RegistrationData, ModelNet40Data


## Ejemplo de Uso

In [6]:
batch_size = 5 #32
workers = 4 #number of data loading workers

In [7]:
trainset = RegistrationData('DCP', ModelNet40Data(train=True))
testset = RegistrationData('DCP', ModelNet40Data(train=False))
train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True, drop_last=True, num_workers=workers)
test_loader = DataLoader(testset, batch_size=batch_size, shuffle=False, drop_last=False, num_workers=workers)

In [8]:
emb_dims = 512 #dim. of the feature vector (default: 512)

In [9]:
dgcnn = DGCNN(emb_dims=emb_dims)
model = DCP(feature_model=dgcnn, cycle=True)
model = model.to(device)

In [10]:
pretrained = "./PreTrainedModels/DCP/best_model.t7"

In [11]:
temp = torch.load(pretrained, map_location=device)
model.load_state_dict(temp, strict=False)

_IncompatibleKeys(missing_keys=[], unexpected_keys=['pointer.model.encoder.layers.0.sublayer.0.norm.norm.weight', 'pointer.model.encoder.layers.0.sublayer.0.norm.norm.bias', 'pointer.model.encoder.layers.0.sublayer.0.norm.norm.running_mean', 'pointer.model.encoder.layers.0.sublayer.0.norm.norm.running_var', 'pointer.model.encoder.layers.0.sublayer.0.norm.norm.num_batches_tracked', 'pointer.model.encoder.layers.0.sublayer.1.norm.norm.weight', 'pointer.model.encoder.layers.0.sublayer.1.norm.norm.bias', 'pointer.model.encoder.layers.0.sublayer.1.norm.norm.running_mean', 'pointer.model.encoder.layers.0.sublayer.1.norm.norm.running_var', 'pointer.model.encoder.layers.0.sublayer.1.norm.norm.num_batches_tracked', 'pointer.model.encoder.norm.norm.weight', 'pointer.model.encoder.norm.norm.bias', 'pointer.model.encoder.norm.norm.running_mean', 'pointer.model.encoder.norm.norm.running_var', 'pointer.model.encoder.norm.norm.num_batches_tracked', 'pointer.model.decoder.layers.0.sublayer.0.norm.norm

### Test

igt: intial ground truth. Used to calculate the accuracy of the model.

- R_ab: Rotation from A to B
- R_ba: Rotation from B to A
- translation ab: translation from A to B
- translation ba: translation from B to A

In [13]:
def get_transformations(igt):
	R_ba = igt[:, 0:3, 0:3]                             # Ps = R_ba * Pt
	translation_ba = igt[:, 0:3, 3].unsqueeze(2)        # Ps = Pt + t_ba
	R_ab = R_ba.permute(0, 2, 1)                        # Pt = R_ab * Ps
	translation_ab = -torch.bmm(R_ab, translation_ba)   # Pt = Ps + t_ab
	return R_ab, translation_ab, R_ba, translation_ba

def display_open3d(template, source, transformed_source=None):
	template_ = o3d.geometry.PointCloud()
	source_ = o3d.geometry.PointCloud()
	template_.points = o3d.utility.Vector3dVector(template)
	source_.points = o3d.utility.Vector3dVector(source + np.array([0,0,0]))
	template_.paint_uniform_color([1, 0, 0])
	source_.paint_uniform_color([0, 1, 0])
	if transformed_source is not None:
		transformed_source_ = o3d.geometry.PointCloud()
		transformed_source_.points = o3d.utility.Vector3dVector(transformed_source)
		transformed_source_.paint_uniform_color([0, 0, 1])
		o3d.visualization.draw_geometries([template_, source_, transformed_source_])
	else:
		o3d.visualization.draw_geometries([template_, source_, transformed_source_])

def test_one_epoch(device, model, test_loader):
	model.eval()
	test_loss = 0.0
	pred  = 0.0
	count = 0
	for i, data in enumerate(tqdm(test_loader)):
		template, source, igt = data
		transformations = get_transformations(igt)
		transformations = [t.to(device) for t in transformations]
		R_ab, translation_ab, R_ba, translation_ba = transformations

		template = template.to(device)
		source = source.to(device)
		igt = igt.to(device)
		# print(type(template), type(source), type(igt))
		print(template.shape, source.shape, igt.shape)
		output = model(template, source)
		display_open3d(template.detach().cpu().numpy()[0], source.detach().cpu().numpy()[0], output['transformed_source'].detach().cpu().numpy()[0])

		identity = torch.eye(3, device=device).unsqueeze(0).repeat(template.shape[0], 1, 1)
		loss_val = torch.nn.functional.mse_loss(torch.matmul(output['est_R'].transpose(2, 1), R_ab), identity) \
			   + torch.nn.functional.mse_loss(output['est_t'], translation_ab[:,:,0])

		cycle_loss = torch.nn.functional.mse_loss(torch.matmul(output['est_R_'].transpose(2, 1), R_ba), identity) \
			   + torch.nn.functional.mse_loss(output['est_t_'], translation_ba[:,:,0])
		loss_val = loss_val + cycle_loss * 0.1

		test_loss += loss_val.item()
		count += 1

	test_loss = float(test_loss)/count
	return test_loss

In [15]:
# result =  test_one_epoch(device, model, test_loader)

In [228]:
# result

## Con datos propios

In [16]:
from torch.utils.data import Dataset, DataLoader

In [17]:
import os

class OwnDataset(Dataset):
    def __init__(self):
        self.init_data(template_path="../templateData/", 
                       source_path="../sourceData/")
        self.n_samples = len(self.template)

    def init_data(self, template_path, source_path):
        self.template = []
        self.source = []
        for filename in os.listdir(template_path):
            if filename.endswith(".ply"):
                pcd = self.read_as_tensor(template_path+filename)

                self.template.append(pcd)
        for filename in os.listdir(source_path):
            if filename.endswith(".ply"):
                pcd = self.read_as_tensor(source_path+filename)
                self.source.append(pcd)

    def __getitem__(self, index):
        return self.template[index], self.source[index]
    
    def __len__(self):
        return self.n_samples

    def as_tensor(self, data):
        return torch.tensor(data, dtype=torch.float32)

    def read_as_tensor(self, path):
        pcd = np.asarray(o3d.io.read_point_cloud(path).points)
        downsample = np.random.randint(0, len(pcd), 1024)
        pcd = pcd[downsample]
        return self.as_tensor(pcd)
    


In [27]:
dataset = OwnDataset()
dataLoader = DataLoader(dataset, batch_size=1, shuffle=False)

In [28]:
template, source = next(iter(dataLoader))
print(template.shape, source.shape)

torch.Size([1, 1024, 3]) torch.Size([1, 1024, 3])


In [29]:
output = model(template, source)
source_transform = output['transformed_source'].detach().cpu().numpy()[0]
source_transform.shape

(1024, 3)

In [30]:
display_open3d(template.detach().cpu().numpy()[0], source.detach().cpu().numpy()[0], source_transform)

In [36]:
import open3d as o3d
template = o3d.io.read_point_cloud("../templateData/bottle_left.ply")

transformed_source_ = o3d.geometry.PointCloud()
transformed_source_.points = o3d.utility.Vector3dVector(source_transform)
transformed_source_.paint_uniform_color([0, 0, 1])

o3d.visualization.draw_geometries([transformed_source_, template])
